In [38]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from scipy import stats
from IPython.display import display
import datetime

def export(df: pd.DataFrame, path_no_dot: str, index=True):
    df.to_csv(f'exports/{path_no_dot}.csv', index=index)
    df.to_excel(f'exports/{path_no_dot}.xlsx', index=index)

    
# THINGS TO CHANGE WHEN ADDING NEW CONTENT

# coloring and sectioning the ELO graph
vrect_colors = ['green', 'red', 'yellow', 'blue', 'orange']
seasons = ['2021_2s', '2022_1s', '2022_2s', '2023_1s','2023_RS']
season_labels = ['2021<br>Doubles', '2022<br>Singles', '2022<br>Doubles', '2023<br>Singles','2023<br>Regular<br>Season']

In [39]:
matches_table.columns

Index(['date', 'event', 'match_number', 'fe1', 'fe2', 't1p1', 't1p2', 't2p1',
       't2p2', 'score1', 'score2'],
      dtype='object')

In [40]:
# data initialization

# needs ppt_analysis.ipynb to be ran first
players_table = pd.read_csv('data/players.csv')
matches_table = pd.read_csv('data/matches.csv').sort_values(['date', 'match_number']).reset_index(drop=True)
matches_table = matches_table.loc[:,matches_table.columns != 'notes']

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(matches_table)

,date,event,match_number,fe1,fe2,t1p1,t1p2,t2p1,t2p2,score1,score2
0,2021-12-04,2021_2s,1.0,False,False,Rose Roché,Aaron Carter,Brian Tafazoli,Evan Sooklal,6,3
1,2021-12-04,2021_2s,2.0,False,False,Coby Lovelace,Jason Jackson,Jack Massingill,Will Simpson,6,3
2,2021-12-04,2021_2s,3.0,False,False,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,3
3,2021-12-04,2021_2s,4.0,False,False,Rohan Chowla,Levin Lee,Paul Bartenfeld,Leah Baetcke,6,0
4,2021-12-04,2021_2s,5.0,True,True,Brian Tafazoli,Evan Sooklal,Jack Massingill,Will Simpson,4,6
...,...,...,...,...,...,...,...,...,...,...,...
97,2023-05-04,2023_RS,NaN,NaN,NaN,Aaron Carter,Will Simpson,Cassie Deering,Roman Ramirez,6,1
98,2023-05-04,2023_RS,NaN,NaN,NaN,Aaron Carter,Will Simpson,Nathan Snow,Leah Baetcke,6,4
99,2023-05-04,2023_RS,NaN,NaN,NaN,Aaron Carter,Will Simpson,Paul Bartenfeld,Coby Lovelace,6,3
100,2023-05-04,2023_RS,NaN,NaN,NaN,Aaron Carter,Will Simpson,Ann Hall,Coby Lovelace,6,4


In [41]:
def expected_score(ratingA, ratingB): # expected difference in cups hit
    # sigmoid vertical bounds
    yrange = 12
    ymin = 6
    # game variables
    expected_cups = 2
    normal_elo_difference = 400

    # score is the expected difference in cups scored,
        # if this function spits out -5, then players of those two ratings should end in a difference of 5 cups
    # this function is a sigmoid function from -6 to 6
    # this function says an elo difference of "normal_elo_difference" should result in absolute difference in "expected_cups"
        # this works: an elo difference of 400 should result in a score difference of 2
    return (yrange / (1 + np.power(expected_cups, (ratingB - ratingA) / normal_elo_difference))) - ymin

def rating_change(score, expected_score):
    # the k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, but with few games, we want to change it frequently
    K = 12
    return K * (score - expected_score)

def expected_score_inverse(expected_cups):
    yrange = 12
    ymin = -6
    normal_elo_change = 400
    # log base 2 shows difference in 400 rating make a score difference of 2
    expected_difference_in_elo = normal_elo_change * np.log2((yrange / (expected_cups - ymin)) - 1)
    return expected_difference_in_elo

# ELO INITIALIZATION
starting_elo = 1200.0
elo = dict()
prev_elo_time = np.empty([len(players_table['player']), len(matches_table) + 1])
elo_time = np.zeros([len(players_table['player']), len(matches_table) + 1])
d_elo_time = np.empty([len(players_table['player']), len(matches_table) + 1])
ec_time = np.empty([len(players_table['player']), len(matches_table) + 1])
caa_time = np.empty([len(players_table['player']), len(matches_table) + 1])

for player in players_table['player']:
    elo.update({player: starting_elo})
    prev_elo_time[players_table['player'][players_table['player'] == player].index[0], 0] = starting_elo
    elo_time[players_table['player'][players_table['player'] == player].index[0], 0] = starting_elo
    caa_time[players_table['player'][players_table['player'] == player].index[0], 0] = starting_elo

# ALGORITHM
prev_t1_row = 1
prev_t2_row = 1

for (i, row) in matches_table.iterrows():
    elos_t1 = list()
    elos_t2 = list()

    elos_t1.append(elo[row.t1p1])
    elos_t2.append(elo[row.t2p1])

    # doubles
    if (type(row.t1p2) == str) and (type(row.t2p2) == str):
        elos_t1.append(elo[row.t1p2])
        elos_t2.append(elo[row.t2p2])
        
        prev_elo_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = elo[row.t1p2]
        prev_elo_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = elo[row.t2p2]

    elo_t1 = np.mean(elos_t1)
    elo_t2 = np.mean(elos_t2)

    # NEW CODE
    prev_elo_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = elo[row.t1p1]
    prev_elo_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = elo[row.t2p1]

    win_prob_t1 = expected_score(elo_t1, elo_t2)
    win_prob_t2 = expected_score(elo_t2, elo_t1)

    rating_change_t1p1 = rating_change(row.score1 - row.score2, win_prob_t1)
    rating_change_t2p1 = rating_change(row.score2 - row.score1, win_prob_t2)

    elo[row.t1p1] += rating_change_t1p1
    elo[row.t2p1] += rating_change_t2p1
    
    d_elo_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = rating_change_t1p1
    d_elo_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = rating_change_t2p1
    
    ec_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = expected_score(elo_t1, elo_t2)
    ec_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = expected_score(elo_t2, elo_t1)

    caa_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = expected_score(elo_t1, starting_elo)
    caa_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = expected_score(elo_t2, starting_elo)

    elo_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = elo[row.t1p1]
    elo_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = elo[row.t2p1]

    # doubles
    if (type(row.t1p2) == str) and (type(row.t2p2) == str):

        rating_change_t1p2 = rating_change(row.score1 - row.score2, win_prob_t1)
        rating_change_t2p2 = rating_change(row.score2 - row.score1, win_prob_t2)

        elo[row.t1p2] += rating_change_t1p2
        elo[row.t2p2] += rating_change_t2p2

        d_elo_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = rating_change_t1p2
        d_elo_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = rating_change_t2p2

        ec_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = expected_score(elo_t1, elo_t2)
        ec_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = expected_score(elo_t2, elo_t1)

        caa_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = expected_score(elo_t1, starting_elo)
        caa_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = expected_score(elo_t2, starting_elo)
        
        elo_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = elo[row.t1p2]
        elo_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = elo[row.t2p2]

players_table['current_elo'] = players_table['player'].map(elo)

player_elo_table = players_table[['player', 'current_elo']].sort_values('current_elo', ascending=False).reset_index(drop=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(player_elo_table)

export(player_elo_table, 'player_elo_table', False)

,player,current_elo
0,Aaron Carter,1537.121991
1,Will Simpson,1396.331534
2,Levin Lee,1391.624867
3,Rohan Chowla,1277.890315
4,Kevin Cooper,1271.813140
5,Roman Ramirez,1270.649156
6,Gabe Silverstein,1233.467836
7,Jack Massingill,1228.341893
8,Nathan Snow,1219.740686
9,Kristian Banlaoi,1216.037582


In [42]:
print("Expected score between 1400 and 1000", expected_score(1400, 1000))
print()
print("A much higher ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1400, 1000)))
print("A much higher ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1400, 1000)))
print("A much higher ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1400, 1000)))
print("A much higher ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1400, 1000)))
print()
print("A closely higher ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1250, 1200)))
print("A closely higher ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1250, 1200)))
print("A closely higher ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1250, 1200)))
print("A closely higher ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1250, 1200)))
print()
print("A closely lower ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1200, 1250)))
print("A closely lower ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1200, 1250)))
print("A closely lower ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1200, 1250)))
print("A closely lower ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1200, 1250)))
print()
print("A much lower ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1000, 1400)))
print("A much lower ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1000, 1400)))
print("A much lower ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1000, 1400)))
print("A much lower ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1000, 1400)))
print()

Expected score between 1400 and 1000 2.0

A much higher ELO, B barely wins: A's change is -36.0
A much higher ELO, A barely wins: A's change is -12.0
A much higher ELO, B hugely wins: A's change is -84.0
A much higher ELO, A hugely wins: A's change is 36.0

A closely higher ELO, B barely wins: A's change is -15.117212459956022
A closely higher ELO, A barely wins: A's change is 8.882787540043978
A closely higher ELO, B hugely wins: A's change is -63.11721245995602
A closely higher ELO, A hugely wins: A's change is 56.88278754004398

A closely lower ELO, B barely wins: A's change is -8.882787540043967
A closely lower ELO, A barely wins: A's change is 15.117212459956033
A closely lower ELO, B hugely wins: A's change is -56.882787540043964
A closely lower ELO, A hugely wins: A's change is 63.117212459956036

A much lower ELO, B barely wins: A's change is 12.0
A much lower ELO, A barely wins: A's change is 36.0
A much lower ELO, B hugely wins: A's change is -36.0
A much lower ELO, A hugely 

In [43]:
prev_elo_time_table = pd.concat([players_table.player, pd.DataFrame(prev_elo_time)], axis=1).T
prev_elo_time_table.columns = players_table.player
prev_elo_time_table = prev_elo_time_table.iloc[2:,:]

elo_time_table = pd.concat([players_table.player, pd.DataFrame(elo_time)], axis=1).T
elo_time_table.columns = players_table.player
elo_time_table = elo_time_table.iloc[2:,:]
elo_time_table.replace(0.0, np.nan, inplace=True)

d_elo_time_table = pd.concat([players_table.player, pd.DataFrame(d_elo_time)], axis=1).T
d_elo_time_table.columns = players_table.player
d_elo_time_table = d_elo_time_table.iloc[2:,:]

ec_time_table = pd.concat([players_table.player, pd.DataFrame(ec_time)], axis=1).T
ec_time_table.columns = players_table.player
ec_time_table = ec_time_table.iloc[2:,:]

caa_time_table = pd.concat([players_table.player, pd.DataFrame(caa_time)], axis=1).T
caa_time_table.columns = players_table.player
caa_time_table = caa_time_table.iloc[2:,:]

# adding current elo to table and sorting the table!!!
# remember that this is now sorted so all slicing will have to remember that it's sorted
sorted_players_table = players_table.copy()
sorted_players_table['current_elo'] = sorted_players_table['player'].map(elo)
sorted_players_table.sort_values('current_elo', inplace=True, ascending=False)
sorted_players_table.reset_index(drop=True, inplace=True)


with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(elo_time_table)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(d_elo_time_table)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(caa_time_table)

export(elo_time_table, 'elo_time_table', True)

player,Kristian Banlaoi,Kevin Cooper,Rohan Chowla,Rose Roché,Aaron Carter,Roman Ramirez,Coby Lovelace,Paul Bartenfeld,Jason Jackson,Jack Massingill,Evan Sooklal,Leah Baetcke,Gabe Silverstein,Reagan Fryatt,Carla Betancourt,Levin Lee,Cason Duszak,Will Simpson,Ann Hall,Helen Dunn,Noah Dale,Yvonne Nguyen,Anna Brown,Brian Tafazoli,Sam Carswell-Tellis,Nathan Snow,Piper Parker,Matthew Rusten,Cassie Deering
1,NaN,NaN,NaN,1236.0,1236.000000,NaN,NaN,NaN,NaN,NaN,1164.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1164.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,1236.000000,NaN,1236.0,1164.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1164.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1236.0,NaN,NaN,NaN,1236.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1164.000000,NaN,NaN,NaN,1164.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1272.0,NaN,NaN,NaN,NaN,1128.000000,NaN,NaN,NaN,1128.000000,NaN,NaN,NaN,1272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1188.0,1140.0,NaN,NaN,NaN,NaN,NaN,NaN,1188.000000,NaN,NaN,NaN,NaN,NaN,1140.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,NaN,NaN,NaN,NaN,1440.937492,1320.599412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1300.147035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1058.266119
99,NaN,NaN,NaN,NaN,1455.524159,NaN,NaN,NaN,NaN,NaN,NaN,1188.642713,NaN,NaN,NaN,NaN,NaN,1314.733701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1219.740686,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,1475.175154,NaN,1210.880076,986.630688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1334.384697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,1487.171735,NaN,1198.883495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1346.381278,1198.228993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


player,Kristian Banlaoi,Kevin Cooper,Rohan Chowla,Rose Roché,Aaron Carter,Roman Ramirez,Coby Lovelace,Paul Bartenfeld,Jason Jackson,Jack Massingill,Evan Sooklal,Leah Baetcke,Gabe Silverstein,Reagan Fryatt,Carla Betancourt,Levin Lee,Cason Duszak,Will Simpson,Ann Hall,Helen Dunn,Noah Dale,Yvonne Nguyen,Anna Brown,Brian Tafazoli,Sam Carswell-Tellis,Nathan Snow,Piper Parker,Matthew Rusten,Cassie Deering
1,0.0,0.0,0.0,36.0,36.0,0.0,0.0,0.0,0.0,0.0,-36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-36.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1221.754931,1221.754931,0.0,36.0,0.0,36.0,-36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,36.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,1201.35646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1201.35646,-36.0,0.0,0.0,0.0,-36.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,72.0,0.0,0.0,0.0,1212.0,-72.0,1212.0,0.0,0.0,-72.0,0.0,0.0,0.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1175.336217,1225.552392,0.0,0.0,1175.336217,0.0,0.0,0.0,24.0,-24.0,0.0,0.0,0.0,0.0,1225.552392,0.0,24.0,0.0,0.0,0.0,0.0,0.0,-24.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.0,0.0,0.0,0.0,55.650411,-55.650411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.650411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-55.650411
99,0.0,0.0,0.0,0.0,14.586667,0.0,0.0,0.0,0.0,0.0,0.0,-14.586667,0.0,0.0,0.0,0.0,0.0,14.586667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-14.586667,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,19.650996,0.0,-19.650996,-19.650996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.650996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,11.996581,0.0,-11.996581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.996581,-11.996581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


player,Kristian Banlaoi,Kevin Cooper,Rohan Chowla,Rose Roché,Aaron Carter,Roman Ramirez,Coby Lovelace,Paul Bartenfeld,Jason Jackson,Jack Massingill,Evan Sooklal,Leah Baetcke,Gabe Silverstein,Reagan Fryatt,Carla Betancourt,Levin Lee,Cason Duszak,Will Simpson,Ann Hall,Helen Dunn,Noah Dale,Yvonne Nguyen,Anna Brown,Brian Tafazoli,Sam Carswell-Tellis,Nathan Snow,Piper Parker,Matthew Rusten,Cassie Deering
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1216.642981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1074.236875,0.0,0.0,0.0
2,0.0,0.0,0.0,0.187089,0.187089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1311.019569,0.0,0.0,0.074051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.074051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1457.154211,0.0,0.0,0.0,0.0,0.0,0.0,1201.177264,0.0,0.0,0.0,0.0,0.0,1310.627617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1229.841243,0.0,0.0
5,0.0,0.129366,-0.129366,0.0,0.0,0.129366,0.0,1211.662298,983.577602,-0.187089,-0.187089,0.0,0.0,0.0,0.0,-0.129366,0.0,-0.187089,1330.285064,0.0,0.0,0.0,0.0,-0.187089,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.0,0.0,0.0,0.0,0.595312,0.23425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1446.080393,0.0,0.595312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.23425
99,0.0,0.0,0.0,0.0,0.880185,0.0,0.0,0.0,0.0,0.0,0.0,0.097613,0.0,0.0,0.0,1389.83627,0.0,0.880185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.097613,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.954242,0.0,-0.423468,-0.423468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.954242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,1.053537,0.0,0.054858,0.0,0.0,1219.96018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.053537,0.054858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1193.14414,0.0,0.0


In [44]:
import plotly.graph_objects as go
import plotly.colors as pc

average_elo = 1200
cup_diffs = [-2, -1, -0.5, -0.25, 0, 0.25, 0.5, 1, 2]
expected_elos = [average_elo - expected_score_inverse(x) for x in cup_diffs]

fig = go.Figure()

# CAA is cups against average (like WAR)
# how many cups will you make against the average player?

hrect_colors = pc.sequential.Turbo
hrect_annots = ['-1.00 CAA','-0.50 CAA','-0.25 CAA','0.00 CAA','+0.25 CAA','+0.50 CAA', '+1.00 CAA', '+2.00 CAA']
for i in range(len(expected_elos) - 1):
    fig.add_hrect(
        annotation_position="top left",
        y0=expected_elos[i],
        y1=expected_elos[i+1],
        fillcolor=hrect_colors[i % len(hrect_colors)],
        opacity=0,
        line_width=0,
        annotation=dict(
            text=f'<b>{hrect_annots[i]}</b>',
            font=dict(
                size=7
            )
        )
    )

ranking = 1
trace_colors = pc.qualitative.Light24
for (i, (player,current_rating)) in enumerate(sorted(elo.items(), key=lambda x:x[1], reverse=True)):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        name=f'#{ranking} ({current_rating:.0f}) {player}',
        mode='lines+markers',
        connectgaps=True,
        text=[f"<br><b>Before-Game ELO: </b>{x[2]:.0f}<br><b>Expected Cups Against Opponent: </b>{x[0]:+.2f}<br><b>Expected Cups Against Average: </b>{x[1]:+.2f}<br><br><b>Change in ELO:</b> {x[3]:+.0f}<br>" for x in zip(
                ec_time_table[player],
                caa_time_table[player],
                prev_elo_time_table[player],
                d_elo_time_table[player]
            )
        ],
        # text=[f"<b>Change in ELO:</b> {c:.0f}<br>" for c in d_elo_time_table[player]],
        line=dict(
            shape = 'hv',
            color=trace_colors[i % len(trace_colors)]
        )
        # maybe add a per-match section of expected score diff
    ))
    ranking += 1

for (i, season) in enumerate(seasons):
    fig.add_vrect(
        annotation_text=season_labels[i],
        annotation_position="top left",
        x0=matches_table['event'][matches_table['event'] == season].index[0] + 0.5,
        x1=matches_table['event'][matches_table['event'] == season].index[-1] + 1.5,
        fillcolor=vrect_colors[i % len(vrect_colors)],
        opacity=0.1,
        line_width=0,
    )

dropdown_labels = [
    "All",
    "2019 Season",
    "2020 Season",
    "2021 Season",
    "2022 Season",
    "Snares",
    "Tenors",
    "Basses",
    "Cymbals",
    "Pit/Cymbals",
    "2022 Grads",
    "2023 Grads",
    "2024 Grads",
    "2025 Grads",
    "2026 Grads",
    "2021 Singles",
    "2022 Doubles",
    "2022 Singles",
    "2023 Doubles"
]
dropdown_categories = [
    [True for _ in sorted_players_table['player']],
    [x != "None" for x in sorted_players_table['2019']],
    [x != "None" for x in sorted_players_table['2020']],
    [x != "None" for x in sorted_players_table['2021']],
    [x != "None" for x in sorted_players_table['2022']],
    [x > 0 for x in sorted_players_table[['2019', '2020', '2021', '2022']].apply(lambda x: x == 'snare').sum(axis=1)],
    [x > 0 for x in sorted_players_table[['2019', '2020', '2021', '2022']].apply(lambda x: x == 'tenors').sum(axis=1)],
    [x > 0 for x in sorted_players_table[['2019', '2020', '2021', '2022']].apply(lambda x: x == 'bass').sum(axis=1)],
    [x > 0 for x in sorted_players_table[['2019', '2020', '2021', '2022']].apply(lambda x: x == 'cymbals').sum(axis=1)],
    [x > 0 for x in sorted_players_table[['2019', '2020', '2021', '2022']].apply(lambda x: x == 'pit-cymbals').sum(axis=1)],
    [x == 2022 for x in sorted_players_table['grad_year']],
    [x == 2023 for x in sorted_players_table['grad_year']],
    [x == 2024 for x in sorted_players_table['grad_year']],
    [x == 2025 for x in sorted_players_table['grad_year']],
    [x == 2026 for x in sorted_players_table['grad_year']],
    [x != "None" for x in sorted_players_table['seed_2021_2s']],
    [x != "None" for x in sorted_players_table['seed_2022_1s']],
    [x != "None" for x in sorted_players_table['seed_2022_2s']],
    [x != "None" for x in sorted_players_table['seed_2023_1s']]
]

def active_ranking(cat):
    i = 0
    t = 0
    retList = list()
    for c in cat:
        retList.append(f"#{i+1} ({sorted_players_table['current_elo'][t]:.0f}) {sorted_players_table['player'][t]}")
        if c is True:
            i += 1
        t += 1
    return retList 

dropdown_dicts = [
    dict(
    label=label,
    method='restyle',
    args=[{"visible": category, "name": active_ranking(category)}]
    ) for (label, category) in zip(dropdown_labels, dropdown_categories)
]


fig.update_layout(
    title=f'<b>The UVA Drumline Pong ELO Rating System by Roman Ramirez</b><br><i>Updated: {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))}<i>',
    xaxis_title='<b>Game Number</b>',
    yaxis_title='<b>ELO Rating</b>',

    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_dicts
            ,
        )       
    ]
)

team1_list = list()
team2_list = list()
for (i, row) in matches_table.iterrows():
    if type(row.t1p2) is not str:
        team1_list.append(row.t1p1)
    else:
        team1_list.append(row.t1p1 + " and " + row.t1p2)

    if type(row.t2p2) is not str:
        team2_list.append(row.t2p1)
    else:
        team2_list.append(row.t2p1 + " and " + row.t2p2)

customdata = np.stack((team1_list, team2_list, matches_table['score1'], matches_table['score2'], matches_table['date']), axis=-1)
hovertemplate = (
    '<i>%{customdata[4]|%A, %B %d, %Y}, Game %{x}</i><br>' +
    '<b>%{fullData.name}</b><br><br>' + 
    '<b>%{customdata[0]} vs. %{customdata[1]}</b><br>' +
    '<b>Final Score:</b> %{customdata[2]}-%{customdata[3]}<br>' + 
    '%{text}' + 
    '<b>After-Game ELO:</b> %{y:,.0f}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Current ELO</i>) <i>Player</i></b>'
)

for (cd, eo) in zip(cup_diffs, expected_elos):
    fig.add_shape(
        type='line',
        x0=elo_time_table.index.min() - 5,
        y0=eo,
        x1=elo_time_table.index.max() + 5,
        y1=eo,
        line=dict(
            color='black',
            dash='dot'
        ),
        opacity=0.10,
    )
    # fig.add_trace(
    #     go.Scatter(
    #         x=[elo_time_table.index.min() - 5],
    #         y=[eo],
    #         text=f"{cd} cups",
    #         mode="text",
    #         textfont=dict(
    #             size=7,
    #         ),
    #         showlegend=False
    #     )
    # )

fig.show()
fig.write_html("index.html")

In [45]:
d_elo_time_table['Aaron Carter']

1             36.0
2      1221.754931
3              0.0
4              0.0
5              0.0
          ...     
98       55.650411
99       14.586667
100      19.650996
101      11.996581
102      49.950255
Name: Aaron Carter, Length: 102, dtype: object

In [46]:
sorted_players_table

,player,grad_year,2019,2020,2021,2022,seed_2021_2s,seed_2022_1s,seed_2022_2s,seed_2023_1s,current_elo
0,Aaron Carter,2023,tenors,tenors,tenors,tenors,1,5,3,8,1537.121991
1,Will Simpson,2023,None,None,pit-cymbals,pit-cymbals,5,None,6,10,1396.331534
2,Levin Lee,2025,None,None,snare,snare,3,None,1,3,1391.624867
3,Rohan Chowla,2025,None,None,snare,snare,3,3,1,2,1277.890315
4,Kevin Cooper,2023,cymbals,cymbals,bass,bass,2,2,2,None,1271.813140
5,Roman Ramirez,2023,bass,bass,bass,bass,2,6,2,1,1270.649156
6,Gabe Silverstein,2025,None,None,cymbals,bass,None,13,5,5,1233.467836
7,Jack Massingill,2024,None,bass,bass,bass,5,10,5,4,1228.341893
8,Nathan Snow,2025,None,None,pit-cymbals,pit-cymbals,None,None,None,12,1219.740686
9,Kristian Banlaoi,2022,cymbals,cymbals,cymbals,None,None,1,None,None,1216.037582


In [47]:
agg_d_elo_table = pd.DataFrame()
agg_d_elo_table['Avg Change in ELO'] = d_elo_time_table.mean(axis=0)
agg_d_elo_table['STD Change in ELO'] = d_elo_time_table.std(axis=0) # how streaky is someone, whats the average difference a change will be from the mean change
agg_d_elo_table.sort_values("STD Change in ELO", ascending=False, inplace=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(pd.DataFrame(agg_d_elo_table))

import plotly.express as px

fig_streakiness = px.bar(
    agg_d_elo_table,
    y='Avg Change in ELO',
    error_y='STD Change in ELO',
    x=agg_d_elo_table.index,
    title="<b>Streakiness</b>",
    )

fig_streakiness.update_layout(
    title=f'<b>Streakiness by Roman Ramirez</b><br><i>Updated: {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))}<i>',
    xaxis_title='<b>Average Change in ELO</b>',
    yaxis_title='<b>Player</b>',
)

fig_streakiness.show()
fig_streakiness.write_html("exports/streakiness.html")

,Avg Change in ELO,STD Change in ELO
player,,
Rohan Chowla,220.182777,493.310123
Levin Lee,187.548343,467.981401
Roman Ramirez,169.535048,445.088594
Aaron Carter,158.086959,406.764209
Coby Lovelace,132.884038,384.301692
Leah Baetcke,134.843769,373.356402
Gabe Silverstein,121.334344,369.425085
Kevin Cooper,110.577029,354.999496
Jack Massingill,107.550439,346.686725
